In [1]:
import joblib
import numpy as np
import pandas as pd

from planet.app import Planet, Geo

In [2]:
model = joblib.load('models/sgd_model.pkl')

In [3]:
def insert_param(frame: pd.Series):
    coord = Geo(frame['place']).coord
    new_data = pd.Series({        
        'Широта': coord['latitude'],
        'Долгота': coord['longitude'],        
    })

    frame = pd.concat([frame, new_data])    
    parametrs = Planet(coord['longitude'], coord['latitude'], frame['date']).get_parametrs()
    
    return pd.concat([frame, pd.Series(parametrs)])

def make_frame(place: str, date: str, period: int) -> pd.DataFrame:

    frame = pd.DataFrame(
        data=pd.date_range(date, periods=period, freq="D").to_list(),
        columns=['date']
    )
    frame['place'] = place
    
    return frame.apply(insert_param, axis=1)

def predict_magnitude(place: str, date: str, period: int):
    frame = make_frame(place, date, period)
    frame['magnitude'] = model.predict(frame.drop(['date', 'place'], axis=1))
    return frame[['date', 'magnitude']]

In [5]:
place = 'Barnaul, Russia'
date = "2023-02-11"  # начальная дата периода
period = 10  # длительность периода  днях

magnitude_frame = predict_magnitude(place, date, period)

In [6]:
magnitude_frame

,date,magnitude
0,2023-02-11,1.877495
1,2023-02-12,1.875179
2,2023-02-13,1.872532
3,2023-02-14,1.869605
4,2023-02-15,1.866513
5,2023-02-16,1.863439
6,2023-02-17,1.860773
7,2023-02-18,1.858562
8,2023-02-19,1.857245
9,2023-02-20,1.856978


In [7]:
magnitude_frame.describe()

,magnitude
count,10.000000
mean,1.865832
std,0.007596
min,1.856978
25%,1.859115
50%,1.864976
75%,1.871800
max,1.877495


In [8]:
magnitude_frame[magnitude_frame['magnitude'] > 2.79]

,date,magnitude
